In [5]:
import numpy as np
import pandas as pd

movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

movies.head()

movies = movies.merge(credits,on='title')

movies.shape

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

movies.head()

movies.dropna(inplace=True)

movies.head()

movies.isnull().sum()

movies.duplicated().sum()

import ast

#ast.literal_eval is used to convert string into list and we can then run loop as list has indices

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

movies.dropna(inplace=True)


import warnings
warnings.filterwarnings('ignore')

movies['genres'].apply(convert)

movies['genres']=movies['genres'].apply(convert)

movies.head()

movies['keywords']=movies['keywords'].apply(convert)

movies.head()

def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
            break
    return L

movies['crew']=movies['crew'].apply(fetch_director)

movies.head()

def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 

movies['cast'] = movies['cast'].apply(convert)
movies.head()

#now overview column is a string and we want to concatenate overview, genre, cast, crew into one column i.e. tags so we need to convert overview col to list 

movies['overview']=movies['overview'].apply(lambda x:x.split())

movies.head()

#now we need to apply one transition that is we need to remove all the spaces from all the columns as for example there is a word Sam Worington and there is another word Sam Mendes now if we dont remove the spaces the tags generated will be Sam, Warington, sam, mendes now if somebody looks for sam the model get confused that which same to show therefore we need to remove spaces and make samWarington as one tag and samMendes as another

movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

movies.head()

movies['tags']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

movies.head()

new_df = movies[['movie_id','title', 'tags']]

new_df.head()

#now we are making this tags column as strings

new_df['tags']=new_df['tags'].apply(lambda x:" ".join(x))

new_df

new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [1]:
#now we are done with data preprocessing ab hme hrr movie ka ek vector bnana h so aise hmare paas 5000 vectors ban jayenge then if koi hmse bolega ek movie k similar movies nukalne ki toh uss movie k vector k closest vale mmovies k vector dedenge..  in order to perform this vectorization we have many ways, we are using the most simpler one i.e. Bag Of Words in this technique hum saare movies k tags ko mila denge let say hmare paas 1000 words bn gye ab hum har movie m check krenge ki vo words kitni baar aa rhe h aise hrr movie k paas 1000D vector ban jayega, Note:- hum stop words ko ignore krenge(aise words jo sentence formation m help krte h like is, an, the, in, etc.)

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [22]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [23]:
vectors.shape

(4806, 5000)

In [24]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [25]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zoo', 'zooeydeschanel', 'zoëkravitz'],
      dtype=object)

In [15]:
#now there are so many words like (action, actions) (activity, activities) which has same meaning just different spellings so now we need to apply steming... stemin is the proces of convertiing words like (love, loved, loving) to (love, love, love) then we remove duplicates we need a library for steming named nltk

In [16]:
!pip install nltk

In [17]:
import nltk

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [19]:
def stem(text):
    y= []
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [20]:
new_df['tags'].apply(stem)

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4804    el mariachi just want to play hi guitar and ca...
4805    a newlyw couple' honeymoon is upend by the arr...
4806    "signed, sealed, delivered" introduc a dedic q...
4807    when ambiti new york attorney sam is sent to s...
4808    ever sinc the second grade when he first saw h...
Name: tags, Length: 4806, dtype: object

In [26]:
#Now we need to calculate dist between these vectors But as we know the curse of dimensionality Euclidean(tip to tip dist of vector y2-y1 ka sq x2-x1 ka sq) fails so instead we will calculate the cosine dist between vectors i.e the angle between the vectors